# Overview
Do model training and evaluation using __*python*__

## Goal 1 - Evaluate Cross Val Model

In [19]:
import cPickle
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from score_auto_gbm.FeatureTransformer import FeatureTransformer
from sklearn.ensemble.partial_dependence import plot_partial_dependence

In [2]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
with open("./score_auto_gbm/gbmFit.pkl", "rb") as pickle_file:
    gbmFit = cPickle.load(pickle_file)

In [4]:
data = pd.read_csv('../data/imports-85.data', header=None)
data.columns = [u'risk', u'normalizedLosses', u'make', u'fuelType', u'aspiration', u'numDoors',
       u'bodyStyle', u'driveWheels', u'engineLocation', u'wheelBase',
       u'length', u'width', u'height', u'curbWeight', u'engineType',
       u'numCylinders', u'engineSize', u'fuelSystem', u'bore', u'stroke',
       u'compressionRatio', u'horsepower', u'peakRPM', u'cityMPG', u'highwayMPG', u'price']
print data.index.size
data.head()

205


,risk,normalizedLosses,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [5]:
trainData = data.ix[:,2:]

trainData.head()

,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,length,width,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [18]:
predictedActualTuple = []
for idx, row in enumerate(trainData.to_dict(orient='records')):
    try:
        prediction = list(gbmFit.predict(pd.DataFrame([row])))[0]
    except ValueError:
        continue
    
    if is_number(data.ix[idx,0]) == True and is_number(prediction) == True:
        predictedActualTuple.extend([(float(prediction), float(data.ix[idx,0]))])

numberPredictions = len(predictedActualTuple)
stdPrediction = sum(map(lambda errorTuple: errorTuple[1], predictedActualTuple))/numberPredictions
squaredError = map(lambda errorTuple: (errorTuple[0] - errorTuple[1])**2, predictedActualTuple)
stdSquaredError = map(lambda errorTuple: (stdPrediction - errorTuple[1])**2, predictedActualTuple)
rmse = (sum(squaredError)/numberPredictions)**0.5
std_rmse = (sum(stdSquaredError)/numberPredictions)**0.5
maxPred, maxAct = reduce(lambda x, y:  x if abs(x[0]-x[1]) > abs(y[0]-y[1]) else y, predictedActualTuple)

print 
('\nThe population size was: %d.\nThe squaured error was: %f.\nRMSE: %f.\nMaxed squared error: %f.\nPredicted & Acutal for max diff: %f & %f.\nSTD RMSE: %f' 
% (numberPredictions,sum(squaredError),rmse, max(squaredError), maxPred, maxAct, std_rmse))



The population size was: 195.
The squaured error was: 10.487928.
RMSE: 0.231914.
Maxed squared error: 3.999332.
Predicted & Acutal for max diff: 0.999833 & -1.000000.
STD RMSE: 1.227453


In [7]:
test = []
test.extend([(0.99,1.00), (1.01,0.98)])
testSquareError = map(lambda x: (x[1] - x[0])**2,test)
print test
print testSquareError
print sum(testSquareError)
print round(float(reduce(lambda x,y: float(x+y), testSquareError)), 10)

[(0.99, 1.0), (1.01, 0.98)]
[0.00010000000000000018, 0.0009000000000000016]
0.001
0.001


In [8]:
# Print troubled rows. They are the ones with ? in them.
# Then why does
trainData.ix[[9,44,45,55,56,58,129,130,131],:]

,make,fuelType,aspiration,numDoors,bodyStyle,driveWheels,engineLocation,wheelBase,length,width,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRPM,cityMPG,highwayMPG,price
9,audi,gas,turbo,two,hatchback,4wd,front,99.5,178.2,67.9,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?
44,isuzu,gas,std,two,sedan,fwd,front,94.5,155.9,63.6,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
45,isuzu,gas,std,four,sedan,fwd,front,94.5,155.9,63.6,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
55,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,70,4bbl,?,?,9.4,101,6000,17,23,10945
56,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,70,4bbl,?,?,9.4,101,6000,17,23,11845
58,mazda,gas,std,two,hatchback,rwd,front,95.3,169.0,65.7,...,80,mpfi,?,?,9.4,135,6000,16,23,15645
129,porsche,gas,std,two,hatchback,rwd,front,98.4,175.7,72.3,...,203,mpfi,3.94,3.11,10.0,288,5750,17,28,?
130,renault,gas,std,four,wagon,fwd,front,96.1,181.5,66.5,...,132,mpfi,3.46,3.90,8.7,?,?,23,31,9295
131,renault,gas,std,two,hatchback,fwd,front,96.1,176.8,66.6,...,132,mpfi,3.46,3.90,8.7,?,?,23,31,9895


## Goal 2 - Identify contributing features
This article calls out a few techniques like _feature importance_ and _partial

#### Get model from the pipeline. Can use index or named_steps per [this article](http://stackoverflow.com/questions/28822756/getting-model-attributes-from-scikit-learn-pipeline)

In [11]:
gbm = gbmFit.steps[1][1]

### Use __*feature importance*__ to analyze effectiveness.

In [12]:
importantFeatures = zip(trainData.columns, gbm.feature_importances_)
importantFeatures.sort(key=lambda x:-x[1])
importantFeatures

[(u'engineType', 0.15571109118551155),
 (u'fuelType', 0.060118267903031167),
 (u'driveWheels', 0.058627068812926453),
 (u'curbWeight', 0.025824447064762068),
 (u'numDoors', 0.017810109459157425),
 (u'wheelBase', 0.013704405797728712),
 (u'make', 0.012053462066536333),
 (u'length', 0.006637071354084788),
 (u'width', 0.0056641482321591295),
 (u'aspiration', 0.00549890366837696),
 (u'height', 0.0048188934320857199),
 (u'numCylinders', 0.0043726511534183977),
 (u'stroke', 0.002740747864547402),
 (u'engineLocation', 0.0020267456757935942),
 (u'horsepower', 0.0013810985295654773),
 (u'bodyStyle', 0.0011872997888196213),
 (u'peakRPM', 0.00031835359842260824),
 (u'price', 0.00028688828332065297),
 (u'compressionRatio', 0.00011359836786870279),
 (u'engineSize', 0.0),
 (u'fuelSystem', 0.0),
 (u'bore', 0.0),
 (u'cityMPG', 0.0),
 (u'highwayMPG', 0.0)]

__*Display Important Features*__
Use the top five most important based on being order of magnitude.

*To do* Automate how we ID these. Hard coding for now.

In [15]:
trainData[map(lambda x: x[0], importantFeatures)].ix[:,0:7].head()

,engineType,fuelType,driveWheels,curbWeight,numDoors,wheelBase,make
0,dohc,gas,rwd,2548,two,88.6,alfa-romero
1,dohc,gas,rwd,2548,two,88.6,alfa-romero
2,ohcv,gas,rwd,2823,two,94.5,alfa-romero
3,ohc,gas,fwd,2337,four,99.8,audi
4,ohc,gas,4wd,2824,four,99.4,audi


### Use  __*partial dependence*__ to analyze effectiveness.

In [29]:
plot_partial_dependence(gbm, trainData, (0))

ValueError: could not convert string to float: mpfi

### Use leave out flag to evaluate model on trained data

## Goal 3 - Rerun training & Eval to Validate Important Features

In [1]:
import cPickle
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from score_auto_gbm.FeatureTransformer import FeatureTransformer
from score_auto_gbm.PipelinePersister import PipelinePersister
from score_auto_gbm.ModelEvaluator import ModelEvaluator
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from datetime import datetime
import json

In [87]:
def get_clean_data(dirty_dataframe):
    result = pd.DataFrame(dirty_dataframe)
    for col in result.select_dtypes(exclude = ['object']).columns:
        result.drop(result[result[col] == '?'].index, axis=0, inplace=True)
        
    return result

### Train re-eval and persist previous model
In this step we show we can use helpers to do the previous training and evaluation.
We then use the helpers to more easily perform a evaluation on a model with only features IDed as important.

#### Train and persist model using original train data

In [9]:
## Set variables
train_data_path = "../data/train_data.json"
base_model_path = "../models/gbmFit-base_model.pkl"
base_processor_path = "../models/preprocess-base_model.pkl"
eval_data_path = "../data/imports-85.data"

# Get Helpers
base_model_persister = PipelinePersister()
evaluator = ModelEvaluator()

In [50]:
# Get train data
train_data = pd.read_json(train_data_path, orient = "records")
X = train_data.drop('risk', axis=1, inplace=False)
y = np.array(train_data[['risk']])

ValueError: labels ['risk'] not contained in axis

In [107]:
# Train and save the model and the processor (for standardization)
model, preprocessor = base_model_persister.fit(X, y)
base_model_persister.save(model, base_model_path)
base_model_persister.save(preprocessor, base_processor_path)

### Evaluate base model to double-check
Wanna ensure both training and persistence worked as expected

In [108]:
model_base = base_model_persister.load(base_model_path)
processor_base = base_model_persister.load(base_processor_path)

In [112]:
eval_data = pd.read_csv(eval_data_path)
eval_data.columns = [u'risk', u'normalizedLosses', u'make', u'fuelType', u'aspiration', u'numDoors',
       u'bodyStyle', u'driveWheels', u'engineLocation', u'wheelBase',
       u'length', u'width', u'height', u'curbWeight', u'engineType',
       u'numCylinders', u'engineSize', u'fuelSystem', u'bore', u'stroke',
       u'compressionRatio', u'horsepower', u'peakRPM', u'cityMPG', u'highwayMPG', u'price']

## aspiration not in original training set ro remove. also remove target (risk)

#clean_eval_data = get_clean_data(eval_data)
# ++++ TO DO figure out clean data issue. +++++

X_eval = eval_data.drop(['aspiration', 'risk'], axis=1)
y_eval = np.array(eval_data[['risk']])

In [113]:
predicted_vs_actuals = evaluator.get_predict_vs_actuals(model_base, X_eval, y_eval)
predicted_vs_actuals_std = evaluator.get_predict_vs_actuals(model_base, X_eval, y_eval, processor_base)

Some predicions may have failed. # of successful predictions: 204, # of actual values: 194
Some predicions may have failed. # of successful predictions: 204, # of actual values: 194


In [114]:
# With standardized input data
print('No standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals))

# Without standardized input data
print('With standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals_std))

No standardization
(0.23251126030495137, 1.2203329284088813, 194)
With standardization
(0.23251126030495137, 1.2203329284088813, 194)


In [98]:
X_eval.select_dtypes(exclude=['object']).columns

Index([u'wheelBase', u'length', u'width', u'height', u'curbWeight',
       u'engineSize', u'compressionRatio', u'cityMPG', u'highwayMPG'],
      dtype='object')

In [97]:
X_eval.drop(X_eval[X['bore'].map(lambda x:str(x)) == '?'].index, axis=0, inplace=True).head()

AttributeError: 'NoneType' object has no attribute 'head'

In [86]:
preprocessor = FeatureTransformer()
preprocessor.fit(get_clean_data(X))
data_std = preprocessor.transform(X_eval[X_eval.columns[[x in X.columns for x in X_eval.columns]]])

TypeError: invalid type comparison

In [81]:
#X[X['normalizedLosses'] == '?'].head()
print(X_eval.columns[[x not in X.columns for x in X_eval.columns]])
print(len(X_eval.columns))

Index([u'normalizedLosses'], dtype='object')
24


In [103]:
print(X_eval.columns.sort_values())
print(X.columns.sort_values())

Index([u'bodyStyle', u'bore', u'cityMPG', u'compressionRatio', u'curbWeight',
       u'driveWheels', u'engineLocation', u'engineSize', u'engineType',
       u'fuelSystem', u'fuelType', u'height', u'highwayMPG', u'horsepower',
       u'length', u'make', u'normalizedLosses', u'numCylinders', u'numDoors',
       u'peakRPM', u'price', u'stroke', u'wheelBase', u'width'],
      dtype='object')
Index([u'aspiration', u'bodyStyle', u'bore', u'cityMPG', u'compressionRatio',
       u'curbWeight', u'driveWheels', u'engineLocation', u'engineSize',
       u'engineType', u'fuelSystem', u'fuelType', u'height', u'highwayMPG',
       u'horsepower', u'length', u'make', u'numCylinders', u'numDoors',
       u'peakRPM', u'price', u'stroke', u'wheelBase', u'width'],
      dtype='object')


### Evauate Variable importance

#### Ensure we can read the saved models

In [119]:
model_reduced = reduced_model_persister.load(reduced_model_path)
preprocessor_reduced = reduced_model_persister.load(reduced_processor_path)

In [120]:
predicted_vs_actuals = evaluator.get_predict_vs_actuals(model_reduced, X_eval, y_eval)
predicted_vs_actuals_std = evaluator.get_predict_vs_actuals(model_reduced, X_eval, y_eval, preprocessor_reduced)

# With standardized input data
print('No standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals))

# Without standardized input data
print('With standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals_std))

No standardization
(0.27024574101812576, 1.2359941993794519, 204)
With standardization
(0.27024574101812576, 1.2359941993794519, 204)


### Model generation and persisting (reduced)

In [109]:
reduced_model_persister = PipelinePersister()
reduced_model_path = "../models/gbmFit-best6features.pkl"
reduced_processor_path = "../models/preprocess-best6features.pkl"

In [117]:
# Train and save the model and the processor (for standardization)
model_reduced, preprocessor_reduced = reduced_model_persister.fit(
    X[['engineType', 'fuelType', 'driveWheels', 'curbWeight', 'numDoors', 'wheelBase']], y)
#reduced_model_persister.save(model_reduced, reduced_model_path)
#reduced_model_persister.save(preprocessor_reduced, reduced_processor_path)

#### Scrap book testing

In [123]:
reduced_model_persister = PipelinePersister()
model_reduced, preprocessor_reduced = reduced_model_persister.fit(
    X[['engineType', 'fuelType', 'driveWheels']], y)
predicted_vs_actuals = evaluator.get_predict_vs_actuals(model_reduced, X_eval, y_eval)
predicted_vs_actuals_std = evaluator.get_predict_vs_actuals(model_reduced, X_eval, y_eval, preprocessor_reduced)

# With standardized input data
print('No standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals))

# Without standardized input data
print('With standardization')
print(evaluator.get_rmse_summary(predicted_vs_actuals_std))

ValueError: Found array with 0 feature(s) (shape=(124, 0)) while a minimum of 1 is required.